## Vector to Raster: A step-by-step breakdown

This notebook processes a single line vector into a raster file with the 'speed' attribute used as the raster pixel value. </br> In our routine, we use this notebook to convert the final processed road rasters into speed surfaces we can combine with the walking friction surface in the next notebook.

Note that if you have multiple speeds (e.g. for dry, monsoon, and winter seasons) you will need to run this notebook multiple times, once for each speed value

In [47]:
import os, sys
from datetime import datetime

import pprint
from pprint import pprint

import common_rasterio_ops as rast_ops

import numpy as np

import rasterio
from rasterio import features
from rasterio import transform
from rasterio.transform import Affine
from rasterio.mask import mask
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape

File paths

In [48]:
data_dir = r'P:\PAK\\Code\Accessibility\Inputs'
out_dir = r'P:\PAK\\Code\Accessibility\Intermediate_Rasters'
fric_dir = r'P:\PAK\\Code\Accessibility\Friction_Surface'
in_dem = r'P:\PAK\\GEO\Elevation'
in_lc = r'P:\PAK\\GEO\Landcover\ICIMOD'


Projections

In [49]:
dest_crs = 'EPSG:32642' # this is a Pakistani UTM projection, assign correct projection for project area

Load Shapefile of aoi to clip the final data

In [50]:
aoi_pth = r'P:\PAK\GEO\Boundaries\OCHA\pak_admbnda_adm1_ocha_pco_gaul_20181218.shp'

In [51]:
aoi = gpd.read_file(aoi_pth)

In [52]:
aoi = aoi[aoi['ADM1_EN'] == 'Khyber Pakhtunkhwa']
aoi = aoi.to_crs(dest_crs)

In [53]:
# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(20000)

Today's date

In [54]:
today = datetime.today().strftime('%y%m%d')

In [55]:
data_date = '210927'

Parameters to change as per your use case!

In [56]:
res = '31m'

In [57]:
season = 'dry'

### Load reference raster

Load in a raster we are aligning with (generally the Off-Road / walking raster itself) so we can match up exactly to its grid and cell size

In [74]:
# choose the raster with the appropriate resolution

with rasterio.open(
        os.path.join(fric_dir,f'KP_friction_walk_{season}_{data_date}_{res}_masked.tif'), 'r') as src:
    base_array = src.read(1)
    base_idx = src.index
    base_profile = src.meta.copy()
    base_tform = src.transform

In [75]:
base_profile

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': -99999.0,
 'width': 17193,
 'height': 21136,
 'count': 1,
 'crs': CRS.from_epsg(32642),
 'transform': Affine(28.23254382673943, 0.0, 502425.3974356071,
        0.0, -31.766168813716423, 4114847.74964671)}

### Load in road file

In [76]:
rd_pth = r'master_transport_Sep23.gpkg'

In [77]:
# replace with master transport file prepared in Step 1
rd = gpd.read_file(os.path.join(data_dir,rd_pth),driver="GPKG")

In [78]:
# replace with speed column you're using
speed_column = f'{season}_speed'

In [79]:
# clipping the roads by the extent of your reference raster

from shapely.geometry import Polygon, box
bbox = box(*src.bounds)

rds_clip = gpd.clip(rd,bbox)

In [80]:
# Reorder the datafram least to highest, so that the generator and therefore speed values are written similarly, 
# This has the ultimate consequence that higher values overwrite lower values where they overlap

rds_clip = rds_clip.sort_values(by=speed_column)

In [81]:
if rds_clip.crs.is_projected == True:
    None
    print('In a metric projection already')
else:
    rds_clip = rds_clip.to_crs(dest_crs)
    print('Reprojected')

In a metric projection already


### Rasterizing vectors

Run a function to generate all the necessary parameters for our raster outputs based on the extent of the input shapefile

In [83]:
bds, ht, wth, src_tform, dst_tform, shapes, scale_factor_x, scale_factor_y = rast_ops.get_raster_params(rds_clip,src,speed_column)

In [84]:
base_tform


Affine(28.23254382673943, 0.0, 502425.3974356071,
       0.0, -31.766168813716423, 4114847.74964671)

In [85]:
dst_tform

Affine(28.23254382673943, 0.0, 502425.3974356071,
       0.0, -31.766168813716423, 4103189.5656920765)

In [86]:
# New code downsamples the code while rasterizing, saving us major headaches with resampling in memory.

road_rast = features.rasterize(shapes,\
                  out_shape = (base_array.shape[0],\
                               base_array.shape[1]),\
                  transform=dst_tform,
                  all_touched=True,
                  dtype = np.float32)

In [87]:
# check values are of the right magnitude
road_rast[road_rast > 0]

array([ 0.25,  0.25,  0.25, ..., 80.  , 80.  , 20.  ], dtype=float32)

In [88]:
road_rast.shape

(21136, 17193)

### Exporting

In [89]:
export_profile = {
    "driver": "GTiff",
    "dtype": "float32",
    "crs":{'init':'EPSG:32642'},
    "height": base_array.shape[0],
    "width": base_array.shape[1],
    "count":1,
    "nodata":0,
    "transform": dst_tform,
    "compress":'LZW'
}


In [95]:
# Export unmasked version

with rasterio.open(
        os.path.join(out_dir,f'KP_OnRoad_{speed_column}_{today}_{res}_test.tif'), 'w',**export_profile) as dst:
    dst.write(road_rast,indexes=1)

In [91]:
# Crop to KP extent
onroad_speed_final_mask, onroad_speed_mask_tform = rast_ops.clip_in_memory(road_rast,export_profile, aoi.geometry)

In [93]:
# Export masked version

with rasterio.open(
        os.path.join(out_dir,f'KP_OnRoad_{speed_column}_{today}_{res}_masked_test.tif'), 'w',**export_profile) as dst:
    dst.write(onroad_speed_final_mask,indexes=1)